# Section 1: Battery Lifespan Analysis

In this MyST notebook, we use Maximum Likelihood Estimation (MLE) to compare two battery brands' lifespans. We'll see how different assumptions about failure mechanisms (random failures vs. aging/wear) lead us to different statistical models and interpretations.

## Overview

- Problem: Decide which battery brand is the better value given limited lifespan data and different possible failure mechanisms.
- Approach: Fit simple parametric models (exponential vs. gamma with shape k=2) using MLE.
- Outcome: Compare expected lifespans and cost-per-month to make a recommendation.

## Learning Objectives

By the end, you should be able to:
- Select an appropriate distribution for simple lifetime data (exponential vs. gamma).
- Write and maximize a likelihood; compute an analytical MLE for the rate parameter.
- Explain the interpretation of parameters and the PDF vs. likelihood distinction.
- Use scipy.optimize.minimize with a negative log-likelihood.
- Visualize a likelihood function and interpret the MLE.
- Make a decision grounded in the model, and reflect on how assumptions change interpretation.

## Dataset

Two small samples of battery lifespans (in months):
- Brand A: 8 observations
- Brand B: 6 observations

We'll start with basic exploration, then fit models.

## Setup

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import minimize

# Reproducibility and plotting style
np.random.seed(42)
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

## Part 1: Load and Explore the Data

In this section, load the Brand A/B samples and compute simple summaries to build intuition before modeling.

In [ ]:
# Brand A battery lifespans (months)
brand_a_data = np.array([18.2, 24.5, 15.3, 22.1, 28.9, 19.7, 26.3, 21.4])

# Brand B battery lifespans (months)
brand_b_data = np.array([31.2, 28.7, 35.4, 29.8, 33.1, 30.9])

print("Brand A")
print(f"n={len(brand_a_data)}, mean={np.mean(brand_a_data):.2f}, std={np.std(brand_a_data):.2f}")

print("Brand B")
print(f"n={len(brand_b_data)}, mean={np.mean(brand_b_data):.2f}, std={np.std(brand_b_data):.2f}")

### Visualize the data

Look at the histograms for both brands. Complete the TODO to plot Brand B.

In [ ]:
plt.figure(figsize=(12, 5))

# Brand A
plt.subplot(1, 2, 1)
plt.hist(brand_a_data, bins=6, density=True, alpha=0.7, color='skyblue', edgecolor='black')
plt.xlabel('Lifespan (months)')
plt.ylabel('Density')
plt.title('Brand A Battery Lifespans')
plt.grid(True, alpha=0.3)

# Brand B
plt.subplot(1, 2, 2)
# TODO 1: Plot histogram for Brand B (bins=6, density=True, alpha=0.7, use a different color)
plt.hist(_____, bins=_____, density=_____, alpha=_____, color='_____', edgecolor='_____')
plt.xlabel('Lifespan (months)')
plt.ylabel('Density')
plt.title('Brand B Battery Lifespans')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Reflection
- Which brand appears to have longer lifespans on average?
- Does the variability suggest a simple model might be reasonable?

## Part 2: Choose Your Model

Assumptions drive model choice:
- Exponential: constant failure rate (memoryless), appropriate for random failures that don’t depend on age.
- Gamma (k=2): increasing failure rate with age (wear-and-tear), appropriate when aging matters.

Set the model below.

In [ ]:
# TODO 2: Switch to "gamma" to explore an aging/wear model
model_type = "exponential"  # Options: "exponential" or "gamma"
model_type

## Part 3: Calculate the MLE Analytically

We estimate the rate parameter lambda for the chosen distribution:
- Exponential: $f(t \mid \lambda) = \lambda e^{-\lambda t}$, MLE $\hat{\lambda} = \frac{n}{\sum_i t_i}$
- Gamma with shape k=2: $f(t \mid \lambda) = \lambda^2 t e^{-\lambda t}$, MLE $\hat{\lambda} = \frac{2n}{\sum_i t_i}$

In [ ]:
def calculate_mle(data, model="exponential"):
    """
    Calculate the MLE for the chosen distribution.
    
    Parameters:
    data: array of observed lifetimes
    model: "exponential" or "gamma"
    
    Returns:
    param: MLE estimate of rate parameter (lambda)
    """
    n = len(data)
    sum_data = np.sum(data)
    
    if model == "exponential":
        # TODO 3: Compute lambda_hat = n / sum(t_i)
        param = _____
    elif model == "gamma":
        # TODO 4: Compute lambda_hat for gamma(k=2): 2n / sum(t_i)
        param = _____
    return param

# Compute parameters and implied means
param_a = calculate_mle(brand_a_data, model_type)
param_b = calculate_mle(brand_b_data, model_type)

if model_type == "exponential":
    mean_a = 1 / param_a
    mean_b = 1 / param_b
else:
    mean_a = 2 / param_a
    mean_b = 2 / param_b

print(f"lambda_A={param_a:.4f}, mean_A={mean_a:.2f}")
print(f"lambda_B={param_b:.4f}, mean_B={mean_b:.2f}")

Interpretation
- The rate parameter lambda controls the typical timescale: larger lambda means shorter lifetimes.
- Under gamma(k=2), the mean is $k/\lambda = 2/\lambda$, reflecting wear that increases failure rates with age.

## Part 3a: Visualize the Likelihood Function

We plot the log-likelihood across a range of lambda values and mark the MLE. The MLE maximizes the log-likelihood.

In [ ]:
def log_likelihood(param, data, model="exponential"):
    """Calculate log-likelihood for the chosen distribution."""
    if param <= 0:
        return -np.inf
    n = len(data)
    sum_data = np.sum(data)
    
    if model == "exponential":
        # log L = n*log(lambda) - lambda*sum(t_i)
        return n * np.log(param) - param * sum_data
    else:  # gamma with shape=2
        # log L = 2n*log(lambda) + sum(log t_i) - lambda*sum(t_i)
        return 2 * n * np.log(param) + np.sum(np.log(data)) - param * sum_data

# Range for visualization (adjust if needed)
param_range = np.linspace(0.01, 0.15, 200)
log_likes_a = [log_likelihood(p, brand_a_data, model_type) for p in param_range]

plt.figure(figsize=(10, 6))
plt.plot(param_range, log_likes_a, 'b-', linewidth=2, label='Log-Likelihood')

# TODO 5: Add a vertical line at the MLE value (use param_a)
plt.axvline(_____, color='red', linestyle='--', linewidth=2, label=f'MLE = {_____:.4f}')

plt.xlabel('Parameter (lambda, rate)')
plt.ylabel('Log-Likelihood')
plt.title(f'Log-Likelihood for Brand A ({model_type} model)')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

What to notice
- The peak corresponds to the MLE. How sharp is the peak? Sharper peaks indicate more information in the data about the parameter.

## Part 3b: Find MLE using Numerical Optimization

We often minimize the negative log-likelihood because optimizers are built for minimization.

In [ ]:
def neg_log_likelihood(param, data, model="exponential"):
    """Negative log-likelihood for minimization."""
    if param <= 0:
        return np.inf
    # TODO 6: Return -log_likelihood(...) so minimizing equals maximizing log-likelihood
    return _____

result = minimize(neg_log_likelihood, x0=0.05, args=(brand_a_data, model_type), bounds=[(0.001, 1)])
param_mle_numerical = result.x[0]

print(f"Analytical MLE = {param_a:.4f}")
print(f"Numerical MLE  = {param_mle_numerical:.4f}")

## Part 4: Compare Models to Data

Overlay the fitted PDF on the histograms to visually assess fit.

In [ ]:
t_range = np.linspace(0.01, 40, 200)  # avoid zero

if model_type == "exponential":
    pdf_a = param_a * np.exp(-param_a * t_range)
    pdf_b = param_b * np.exp(-param_b * t_range)
else:  # gamma with shape=2
    pdf_a = (param_a**2) * t_range * np.exp(-param_a * t_range)
    pdf_b = (param_b**2) * t_range * np.exp(-param_b * t_range)

plt.figure(figsize=(12, 5))

# Brand A
plt.subplot(1, 2, 1)
plt.hist(brand_a_data, bins=6, density=True, alpha=0.5, color='skyblue', edgecolor='black', label='Data')
plt.plot(t_range, pdf_a, 'b-', linewidth=2, label=f'{model_type.capitalize()} model')
plt.xlabel('Lifespan (months)')
plt.ylabel('Density')
plt.title('Brand A: Data vs Model')
plt.legend()
plt.grid(True, alpha=0.3)

# Brand B
plt.subplot(1, 2, 2)
plt.hist(brand_b_data, bins=6, density=True, alpha=0.5, color='lightcoral', edgecolor='black', label='Data')
# TODO 7: Plot the model PDF for Brand B
plt.plot(t_range, _____, 'r-', linewidth=2, label=f'{model_type.capitalize()} model')
plt.xlabel('Lifespan (months)')
plt.ylabel('Density')
plt.title('Brand B: Data vs Model')
plt.legend()
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

Questions
- Which model appears to overlay the data better? Does it differ by brand?

## Part 5: Make the Decision

We'll compute expected lifespans under the chosen model, convert prices to cost-per-month, and pick the better value.

In [ ]:
price_a = 45
price_b = 65

if model_type == "exponential":
    mean_a = 1 / param_a
    mean_b = 1 / param_b
else:
    mean_a = 2 / param_a
    mean_b = 2 / param_b

cost_per_month_a = price_a / mean_a
# TODO 8: Fill in Brand B cost-per-month
cost_per_month_b = _____

print(f"mean_A={mean_a:.2f}, cost_A=${cost_per_month_a:.2f}/month")
print(f"mean_B={mean_b:.2f}, cost_B=${cost_per_month_b:.2f}/month")

winner = "A" if cost_per_month_a < cost_per_month_b else "B"
print(f"Winner: Brand {winner}")

Interpretation
- The “winner” minimizes dollars per month of expected use.
- Note how model choice can change the expected mean and thus the decision.

## Part 6: Model Comparison (Exponential vs. Gamma)

Now compare the decision under both modeling assumptions.

In [ ]:
models = ["exponential", "gamma"]
results = {}

for model in models:
    pa = calculate_mle(brand_a_data, model)
    pb = calculate_mle(brand_b_data, model)
    if model == "exponential":
        ma, mb = 1/pa, 1/pb
    else:
        ma, mb = 2/pa, 2/pb
    ca, cb = 45/ma, 65/mb
    results[model] = {"mean_a": ma, "mean_b": mb, "cost_a": ca, "cost_b": cb, "winner": "A" if ca < cb else "B"}

for model in models:
    r = results[model]
    print(f"{model.upper()}: mean_A={r['mean_a']:.1f}, cost_A=${r['cost_a']:.2f}/m | mean_B={r['mean_b']:.1f}, cost_B=${r['cost_b']:.2f}/m | winner={r['winner']}")

Insight
- If the winner is consistent across models, your decision is robust to these assumptions.
- If not, you’ve learned that assumptions about failure mechanism materially affect the recommendation.

## Model Validation (Synthetic Demonstration)

We demonstrate that our MLE procedure recovers known parameters on synthetic data.

In [ ]:
np.random.seed(123)

if model_type == "exponential":
    true_param_a = 0.045
    true_param_b = 0.032
    test_data_a = np.random.exponential(1/true_param_a, 50)
    test_data_b = np.random.exponential(1/true_param_b, 50)
else:
    true_param_a = 0.09
    true_param_b = 0.064
    test_data_a = np.random.gamma(2, 1/true_param_a, 50)
    test_data_b = np.random.gamma(2, 1/true_param_b, 50)

test_param_a = calculate_mle(test_data_a, model_type)
test_param_b = calculate_mle(test_data_b, model_type)

print(f"True_A={true_param_a:.4f}, Est_A={test_param_a:.4f}")
print(f"True_B={true_param_b:.4f}, Est_B={test_param_b:.4f}")

## Summary

- We modeled battery lifespans using exponential (random failures) and gamma k=2 (aging/wear).
- We derived and computed MLEs, visualized the likelihood, and confirmed with numerical optimization.
- We translated model parameters into expected lifespans and cost-per-month to make a decision.
- We compared decisions across models to assess robustness.

### Learning Objectives Review

- [x] Identify an appropriate distribution for simple lifetime data.
- [x] Write and maximize a likelihood; compute analytical MLE.
- [x] Explain parameter interpretation and PDF vs. likelihood.
- [x] Use scipy.optimize.minimize on negative log-likelihood.
- [x] Visualize likelihood and interpret the MLE.
- [x] Make a model-informed decision and examine the role of assumptions.

### Reflection

- What observable features might suggest wear (gamma) over memoryless failures (exponential)?
- How might censored or truncated data change the MLE and interpretation?
- If you had more data, what diagnostics would you add (e.g., QQ-plots, AIC/BIC)?